# MNIST-10 Quantum Counting Demo

This notebook trains a small classical model, loads its weights into the
QArgus quantum model, runs inference, and applies a quantum-counting
style estimator to count a target digit.


In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
import sys
from pathlib import Path

ROOT = Path.cwd().parents[0]
SRC = ROOT / 'src'
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
from qargus import QuantumResNet, QuantumResNetConfig, estimate_count, run_semantic_model, model_to_qasm
from qargus.weights import load_sklearn_classifier


In [2]:
mnist = fetch_openml("mnist_784", version=1, as_frame=False)
X = mnist.data.astype(np.float32) / 255.0
y = mnist.target.astype(int)

rng = np.random.default_rng(0)
train_idx = rng.choice(len(X), size=1000, replace=False)
remaining = np.setdiff1d(np.arange(len(X)), train_idx, assume_unique=False)
test_idx = rng.choice(remaining, size=128, replace=False)

X_train = X[train_idx]
y_train = y[train_idx]
X_test = X[test_idx]
y_test = y[test_idx]
X_test_img = X_test.reshape(-1, 1, 28, 28)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


ImportError: Using `parser='auto'` with dense data requires pandas to be installed. Alternatively, explicitly set `parser='liac-arff'`.

In [ ]:
clf = SGDClassifier(
    loss="log_loss",
    fit_intercept=False,
    max_iter=2000,
    tol=1e-3,
    random_state=0,
)
clf.fit(X_train, y_train)

train_acc = np.mean(clf.predict(X_train) == y_train)
test_acc = np.mean(clf.predict(X_test) == y_test)
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")


Train accuracy: 1.000
Test accuracy: 0.836


In [ ]:
config = QuantumResNetConfig(
    input_shape=(1, 28, 28),
    num_classes=10,
    num_blocks=0,
    channels=1,
    kernel_size=1,
    output_mode="classifier",
)
model = QuantumResNet(config)
load_sklearn_classifier(model, clf)

preds = []
quantum_logits = []
for img in X_test_img:
    result = model.forward(img)
    quantum_logits.append(result.logits)
    preds.append(int(np.argmax(result.probabilities)))

target_digit = 3
marks = [p == target_digit for p in preds]
qc = estimate_count(marks, precision_bits=6)

print("Target digit:", target_digit)
print("True count:", qc.true_count)
print("Estimated count:", qc.estimated_count)
print("Oracle queries (simulated):", qc.oracle_queries)


Target digit: 3
True count: 15
Estimated count: 15
Oracle queries (simulated): 63


## Deploy with semantic execution and QASM export

Semantic execution runs the same pipeline without circuit synthesis.
QASM export uses Qiskit/Aer to compile the unitary circuit for a concrete input.


In [ ]:
sample = X_test_img[0]
semantic = run_semantic_model(model, sample)
forward = model.forward(sample)

print('Semantic logits (L2-normalized):', semantic.state[:5])
print('Forward logits (L2-normalized):', forward.logits[:5])
print('Max diff:', np.max(np.abs(semantic.state - forward.logits)))


Semantic logits (L2-normalized): [-0.37091857 -0.2862483  -0.24978778 -0.20342551 -0.44136   ]
Forward logits (L2-normalized): [-0.37091857 -0.2862483  -0.24978778 -0.20342551 -0.44136   ]
Max diff: 0.0


In [ ]:
try:
    qasm = model_to_qasm(model, input_state=sample, include_measurements=False)
    qasm_path = ROOT / 'notebooks' / 'mnist10_semantic_unitary.qasm'
    qasm_path.write_text(qasm)
    print('Wrote QASM to:', qasm_path)
    print(qasm.splitlines()[0])
except ImportError as exc:
    print('QASM export requires qiskit and qiskit-aer:', exc)


QASM export requires qiskit and qiskit-aer: qiskit and qiskit-aer are required for QASM export


In [ ]:
def _l2_normalize_rows(arr):
    norms = np.linalg.norm(arr, axis=1, keepdims=True)
    return arr / np.clip(norms, 1e-12, None)

X_test_norm = _l2_normalize_rows(X_test)
classical_logits_raw = X_test_norm @ clf.coef_.T
classical_logits = _l2_normalize_rows(classical_logits_raw)
classical_preds = np.argmax(classical_logits, axis=1)

agreement = np.mean(classical_preds == np.array(preds))
max_logit_diff = np.max(np.abs(classical_logits - np.vstack(quantum_logits)))
max_raw_diff = np.max(np.abs(classical_logits_raw - np.vstack(quantum_logits)))
print(f"Quantum/classical agreement: {agreement:.3f}")
print(f"Max logit diff (normalized): {max_logit_diff:.6f}")

for i in range(5):
    print(i, classical_preds[i], preds[i])


Quantum/classical agreement: 1.000
Max logit diff (normalized): 0.000000
0 7 7
1 7 7
2 3 3
3 6 6
4 2 2


## Resource estimation and advantage signals

This section summarizes semantic execution metadata, circuit gate counts,
and a classical baseline for comparison.


In [ ]:
sample = X_test_img[0]
semantic = run_semantic_model(model, sample)
forward = model.forward(sample)

print('Semantic uses:', semantic.report.uses)
print('Semantic success prob:', semantic.report.cumulative_success_prob)
print('Ancilla peak (clean, dirty):', semantic.report.ancilla_clean_peak, semantic.report.ancilla_dirty_peak)
print('Trace resource estimate:', forward.trace.resources)

input_dim = int(np.prod(config.input_shape))
classical_macs = input_dim * config.num_classes
print('Classical MACs (linear head):', classical_macs)

if 'qc' in globals():
    print('Classical count cost:', len(preds), 'oracle calls')
    print('Quantum counting oracle calls (simulated):', qc.oracle_queries)

try:
    from qargus import build_regime3_unitary_circuit
    from qiskit import transpile
    from qiskit_aer import Aer

    circuit = build_regime3_unitary_circuit(model)
    backend = Aer.get_backend('aer_simulator')
    transpiled = transpile(circuit, backend=backend, optimization_level=1)
    counts = transpiled.count_ops()
    two_qubit = sum(counts.get(g, 0) for g in ('cx', 'cz', 'swap'))
    print('Circuit depth:', transpiled.depth())
    print('Two-qubit gates:', two_qubit)
    print('Gate counts:', counts)
except ImportError as exc:
    print('Circuit resource estimation requires qiskit and qiskit-aer:', exc)


Semantic uses: 1
Semantic success prob: 1.0
Ancilla peak (clean, dirty): 0 0
Trace resource estimate: ResourceEstimate(ancilla_qubits_clean=0, ancilla_qubits_dirty=0, depth=0, one_qubit_gates=0, two_qubit_gates=0, t_count=0, measurements=0, qram_queries=0, oracle_queries=0, classical_queries=0, postselections=0)
Classical MACs (linear head): 7840
Classical count cost: 128 oracle calls
Quantum counting oracle calls (simulated): 63
Circuit resource estimation requires qiskit and qiskit-aer: No module named 'qiskit_aer'
